# k-NN Collaborative Filtering (Baseline)
The following notebook illustrates our k-NN collaborative filtering approach (uses as our baseline model) that uses track co-occurrence in playlists to recommend tracks to playlists.

## 0. Motivation
In order to recommend relevant songs to playlists, it is natural to think about songs that are in a similar playlist but not currently in our target playlist. 

Our k-NN collaborative filtering modeling approach looks at playlists that share similar tracks (i.e., if the two playlists have a high number of overlapping tracks) and recommend songs that are in the similar playlists but not in the target playlist.

## 0.1 Strategy
To compute prediction set per playlist, our model does the following:,
1. The model finds k nearest neighbor playlists of our target playlist (measured by cosine distance between playlists).

2. From the k-NN playlists, the model then build the recommendation by recommending tracks that are in the neighbor playlists but not in the target playlist, starting from the most similar playlist. It will stop building the recommendation list until it reaches the pre-determined length (test set size * 15 for each playlist)
   

## 1. Data Processing

### 1.1 Train-test split

We did a stratified splitting of the data (by Playlistid) into training and test set by 80-20. Stratified splitting ensures that we have the right proportion of train and test for playlists of different lengths.

In [4]:
train.shape, test.shape

((1970, 28), (616, 28))

### 1.2 Binary Sparse Matrix

We used the training set to create a binary sparse matrix with 100 playlists and 1534 unique songs. Each row represents tracks that are in the playlist (1) or not (0). As you can imagine, it is a sparse matrix, because one playlist only has maximum of 350 songs in our dataset, while we have 1534 unqiue songs in the matrix.

We then transformed the matrix to a Compressed Sparse Row matrix from scipy and fit a 5-nearest-neighbor model (using cosine as distance metric and brute-force search).

In [8]:
co_mat.shape

(100, 1534)

Below shows the few rows of the matrix.

In [6]:
co_mat.head()

Track_uri,spotify:track:00LfFm08VWeZwB0Zlm24AT,spotify:track:00qOE7OjRl0BpYiCiweZB2,spotify:track:01a0J96fRD91VnjQQUCqMK,spotify:track:01iyCAUm8EvOFqVWYJ3dVX,spotify:track:027h5P3kCyktHv9dpHUBBS,spotify:track:02M6vucOvmRfMxTXDUwRXu,spotify:track:03L2AoiRbWhvt7BDMx1jUB,spotify:track:03LpkqucyYKcYclDs8HuxO,spotify:track:03fT3OHB9KyMtGMt2zwqCT,spotify:track:03tqyYWC9Um2ZqU0ZN849H,...,spotify:track:7yFMhCJOsH7khgpdnyrZAZ,spotify:track:7yHEDfrJNd0zWOfXwydNH0,spotify:track:7ySUcLPVX7KudhnmNcgY2D,spotify:track:7yfg0Eer6UZZt5tZ1XdsWz,spotify:track:7yq4Qj7cqayVTp3FF9CWbm,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,spotify:track:7zBPzAjKAqQpcv8F8GCq5s,spotify:track:7zWj09xkFgA9tcV6YhfU6q,spotify:track:7zbq8RT5Kd3ExOGVTiUQbR,spotify:track:7zxRMhXxJMQCeDDg0rKAVo
Playlistid,,,,,,,,,,,,,,,,,,,,,
430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2. Model Performance
### 2.1 Metrics 
We used the following metrics to evaluate our model, based on Spotify RecSys [rules](https://recsys-challenge.spotify.com/rules)

1. **R-precision**: the number of retrieved relevant tracks divided by the number of known relevant tracks (i.e., the number of withheld tracks). This metric rewards total number of retrieved relevant tracks (regardless of order).\n,
2. **NDCG**: Discounted cumulative gain (DCG) measures the ranking quality of the recommended tracks, increasing when relevant tracks are placed higher in the list. Normalized DCG (NDCG) is determined by calculating the DCG and dividing it by the ideal DCG in which the recommended tracks are perfectly ranked.\n,
    

### 2.2 Model Test-Set Performance

| Data | R-Precision | NDCG | Average of the two metrics |
|------|------|------|----- |
|Test set of 100 playlists | 0.077025|0.080346|0.078685|
|Test set of 1000 playlists | | | |

## 3. Conclusion

We achieved a 0.07 R-precision score and 0.08 NDCG score with our baseline k-NN collaborative filtering model. The current model only consider track co-occurence between playlists, we are curious to know if the model will improve if audio features are added. 

In [15]:
col_filter = NearestNeighbors(metric='cosine', algorithm='brute')
col_filter.fit(co_mat_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [9]:
def nholdout(playlist_id, df):
    '''Pass in a playlist id to get number of songs held out in val/test set'''
    
    return len(df[df.Playlistid == playlist_id].Track_uri)

def kpredict(knnmodel, playlist_id, df):
    '''for a playlist id, generate list of 15*k predictions where k is num holdouts''' 
    
    k = nholdout(playlist_id, df)*15 # number of holdouts
    ref_songs = co_mat.columns.values[co_mat.loc[playlist_id] == 1] # songs already in playlist
    dist, ind = knnmodel.kneighbors(np.array(co_mat.loc[playlist_id]).reshape(1, -1), n_neighbors = 99)
    rec_ind = co_mat.index[ind[0]] # recommended playlists
    
    n_pred = 0
    pred = []
    for i in rec_ind:
        new_songs = co_mat.columns.values[co_mat.loc[i] == 1] # potential recommendations
        for song in new_songs:
            if song not in ref_songs: # only getting songs not already in target playlist
                pred.append(song)
                n_pred += 1
                if n_pred == k:
                    break
        if n_pred == k:
            break
    
    return pred

In [16]:
### Prediction Example
pi = 430 # target playlist index
kpreds = kpredict(col_filter, pi, val) # list of predictions

In [17]:
val_set = val[val.Playlistid == pi]
val_set = val_set['Track_uri'] # ground truth

## Metrics

In [19]:
def r_precision(prediction, val_set):
# prediction should be a list of predictions
# val_set should be pandas Series of ground truths
    score = np.sum(val_set.isin(prediction))/val_set.shape[0]
    return score

In [20]:
### Example Usage
r_precision(kpreds, val_set)

0.0

In [26]:
### NDCG Code Source: https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [28]:
### Example Usage
# Generate binary relevance array
r = np.zeros(len(kpreds))
for i, p in enumerate(kpreds):
    if p in val_set:
        r[i] = 1

ndcg_at_k(r, len(r))

0.0

## Baseline Model Performance

In [57]:
rps = []
ndcgs = []
for pid in co_mat.index:
    ps = kpredict(col_filter, pid, val) # predictions
    vs = val[val.Playlistid == pid].Track_uri # ground truth
    rps.append(r_precision(ps, vs))
    
    r = np.zeros(len(ps))
    for i, p in enumerate(ps):
        if np.any(vs.isin([p])):
            r[i] = 1
    ndcgs.append(ndcg_at_k(r, len(r)))
    

In [58]:
avg_rp = np.mean(rps)
avg_ndcg = np.mean(ndcgs)
print('Avg. R-Precision: ', avg_rp)
print('Avg. NDCG: ', avg_ndcg)
print('Total Sum: ', np.mean([avg_rp, avg_ndcg]))

Avg. R-Precision:  0.07702539127539126
Avg. NDCG:  0.08034624710411524
Total Sum:  0.07868581918975326
